In [1]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-20 17:08:15--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-20 17:08:16 (31.3 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [1]:
import minsearch 
import json
from openai import OpenAI

ModuleNotFoundError: No module named 'openai'

In [4]:
with open("documents.json") as f:
    file = json.load(f)

In [ ]:
documents =[]

for course_dict in file:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [ ]:
documents[0]

In [ ]:
index = minsearch.Index(
    text_fields=['question','text','section'],
    keyword_fields=['course']
    )

In [ ]:
index.fit(documents)

In [ ]:
q = "this course alreaydy started.Can  still enroll it ?"

In [ ]:
index.search(query=q,num_results=10,filter_dict={'course':'data-engineering-zoomcamp'})

In [8]:
! pip show openai

Name: openai
Version: 1.35.1
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /workspaces/llmops-zoomcamp/venv/lib/python3.10/site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [ ]:
! echo $PYTHONPATH
